In [1]:
from bs4 import BeautifulSoup as bs
from datetime import datetime
from pandasql import sqldf
import requests as req
import pandas as pd
import sqlite3
import time
import os
import re

In [2]:
NOW     = datetime.now()
Y, M, D = NOW.year, NOW.month, NOW.day

In [3]:
whole_page = ""

start_time = time.time()

for idx in range(5):
    url = f'https://store.steampowered.com/search/?specials=1&filter=topsellers&page={idx}'
    res = req.get(url)
    whole_page += res.text

time.time() - start_time

2.0356216430664062

In [4]:
start_time = time.time()
soup       = bs(whole_page, 'html.parser')
sales      = soup.select('div#search_resultsRows > a') 
time.time() - start_time

10.286655187606812

In [5]:
dbpath = 'steamsale.db'
conn   = sqlite3.connect(dbpath)
cursor = conn.cursor()

In [6]:
query = """
CREATE TABLE IF NOT EXISTS steamsale(
id INTEGER PRIMARY KEY AUTOINCREMENT,
appid INTEGER NOT NULL,
percent INTEGER NOT NULL,
original INTEGER NOT NULL,
discounted INTEGER NOT NULL,
date DATE NOT NULL);"""

cursor.execute(query)

In [7]:
def preprop(string, dtype = 'discount'): 
    
    if dtype == 'discount': string = string.replace(' ', '').split('₩')[-1]
    string = string.replace(',', '').replace('₩', '')
    
    return int(string)
    

sale_list = []
for idx, sale in enumerate(sales):
    try:
        appid      = sale['data-ds-appid']

        percent    = sale.select('.search_discount > span')[0].text
        percent    = int(percent.replace('-', '').replace('%', ''))

        original   = preprop(sale.select('strike')[0].text)
        discounted = preprop(sale.select(".discounted")[0].text)

        today = f'{Y}-{str(M).zfill(2)}-{str(D).zfill(2)}'
        query = f"""
                INSERT INTO steamsale
                (appid, percent, original, discounted, date)
                VALUES({appid}, {percent}, {original}, {discounted}, {today});
                """

        cursor.execute(query)
              
    except Exception as e: print(e)

list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range


In [12]:
today

'2023-01-10'

In [10]:
with conn:
    with open('backup.sql', 'w') as f:
        for line in conn.iterdump():
            f.write('%s\n' % line)
        print('Completed')

Completed


In [15]:
query = f"""
            SELECT *
            FROM steamsale
            WHERE date={today}
        """

cursor.execute(query)
cursor.fetchone()[-1]

2012